# Read NSWFRD 2014
Read the spreadsheet from NSW Flora Fire response database and extract hyperlinks that point to references.
We will use the _openpyxl_ library in ***python***

In [1]:
import openpyxl
from pathlib import Path
import os

In [5]:
repodir = Path("../..") 
inputdir = repodir / "data/"

## Open the workbook and read main spreadsheet
Here we will load the workbook:

In [11]:
wb = openpyxl.load_workbook(inputdir / "NSWFFRDv2.1.xlsx")

List all worksheets:

In [12]:
for k in wb.worksheets:
    print(k)

<Worksheet "Read Me">
<Worksheet "SpeciesData">
<Worksheet "References">
<Worksheet "Notes">
<Worksheet "VA Groups">


In [13]:
ws = wb['Read Me']

In [14]:
ws

<Worksheet "Read Me">

Use the sheet name to read data

In [5]:
ws = wb['SpeciesData']

Let's look at all the values in the second column (Species code)

In [6]:
ws['B2'].value

'Species Code'

We want to count the number of unique values:

In [7]:
row_count = ws.max_row
column_count = ws.max_column
j=2
unique_list = list()
unique_items = 0
for i in range(1, row_count + 1):
    item = ws.cell(row=i, column=j).value
    if item not in unique_list and item is not None:
        unique_list.append(item)
        unique_items += 1
print(unique_items)

3000


Which columns include information on traits? this will print out the names in the second row...:

In [8]:
for j in range(1, column_count + 1):
    print(ws.cell(row=2, column=j).value)


Current Scientific Name
Species Code
Legal Status
Exotic
2010 Update
Notes on Name / Synonym as used in source reference
Family
Group
Life form
Fireresponse
Comments on regeneration
Resprout location
Seed storage
Seed dispersal mechanism
Seed dispersal distance
Seed weight / size
Seed viability
Dormancy
Germination cue
Fecundity
Seed predation
Post-fire recruitment
Establishment
Post-fire flowering
Flowering time
Primary juvenile period
Secondary juvenile period
Seed set
Seed-bank developed
Fire tolerance
Life span
Seed-bank longevity
"Maturity" (from source)
"Extinction" (from source)
"Rec. min fire interval" (from source)
"Rec. max fire interval" (from source)
NC
CC
SC
NT
CT
ST
NWS
CWS
SWS
NWP
SWP
NFWP
SFWP
Distribution: extra NSW
Vegetation
Rainforest
Wet Sclerophyll Forest (Shrubby)
Wet Sclerophyll Forest (Grassy)
Grassy Woodland
Grassland
Dry Sclerophyll Forest (Shrub/Grass)
Dry Sclerophyll Forest (Shrubby)
Heathland
Alpine Complex
Freshwater Wetland
Forested Wetlands
Saline Wetla

In [92]:
ws.cell(row=2, column=31).value

'Life span'

## Dealing with hyperlinks

The cell Q6 has a hyperlink. We can use cell rows and columns or cell name:

In [9]:
type(ws.cell(row=6, column=17).hyperlink)
# same as 
type(ws['Q6'].hyperlink)

openpyxl.worksheet.hyperlink.Hyperlink

If the cell is a hyperlink it will have a value to "display" and will point to a "location" within the workbook: 

In [10]:
ws.cell(row=6, column=17).hyperlink.display

'viability average-very good'

In [11]:
# This will fail if there is no hyperlink 
print(ws.cell(row=6, column=17).hyperlink.location)

References!C94


Let's see the value of this reference:

In [12]:
hlink = ws.cell(row=6, column=17).hyperlink.location
hlink = hlink.split("!")

This gives the name of the target sheet and the corresponding cell. We need to read the cell to its right side (add one to the column number) to get the information we need.

In [13]:
ref = wb[hlink[0]]
print("Cell value is :: " + str(ref[hlink[1]].value))
nlink = ref.cell(row=ref[hlink[1]].row,column=ref[hlink[1]].col_idx + 1)

print("Reference data is :: " + nlink.value) 


Cell value is :: 93
Reference data is :: Mortlock, W. & Lloyd, MV (Eds) (2001) Floradata - A guide to collection, storage and propogation of Australian native plant seed. AUsttralian Centre for Mining Environmental Research, Brisbane; Australian National Botanic Gardens, CSIRO Forestry and Forest Products and Greening Australia Limited, Canberra. Searchable Database February 2001. a=survey data, b=test data


If there is no hyperlink, it will result in NoneType

In [14]:
type(ws.cell(row=5, column=17).hyperlink)

NoneType

In [15]:
type(ws.cell(row=5, column=17))


openpyxl.cell.cell.Cell

In [16]:
ws.cell(row=5, column=17)


<Cell 'SpeciesData'.Q5>

## Read data from a column
For a selected variable (column), we can query data for the list of species.


In [17]:
ws['Q6']

<Cell 'SpeciesData'.Q6>

In [93]:
ws.cell(row=2, column=31).value

'Life span'

In [95]:
ws.cell(row=16, column=31).value

Example loop for querying values from one variable for all species in a range of cells:

In [96]:
i=3
j=31
varname=ws.cell(row=2, column=j).value
print(varname)
for i in range(13,40):
    spname=ws.cell(row=i, column=1).value
    spcode=ws.cell(row=i, column=2).value
    varvalue=ws.cell(row=i, column=j).value
    varref=ws.cell(row=i, column=j).hyperlink
    if varvalue is not None:
        if varref is not None:
            print("%s: %s / %s / %s" % (spcode,spname,varvalue, varref.location))
        else:
            print("%s: %s / %s " % (spcode,spname,varvalue))


Life span
3710: Acacia baileyana / 20-30 / References!C2
3716: Acacia binervata / 20 
3717: Acacia binervia / 50-100 / References!C2
8601: Acacia bulgaensis / c. 20-40 
3747: Acacia constablei / short? / References!N16


## Read data for a row
We can now do the same for a single species (row) and query values of each variable in a range. For example:

In [21]:
# for j in 17
i=18
spname=ws.cell(row=i, column=1).value
spcode=ws.cell(row=i, column=2).value
print("%s: %s" %(spcode,spname))
    
for j in range(3,30):
    varname=ws.cell(row=2, column=j).value
    varvalue=ws.cell(row=i, column=j).value
    varref=ws.cell(row=i, column=j).hyperlink
    if varvalue is not None:
        if varref is not None:
            print("%s: %s / %s / %s" % (j,varname,varvalue, varref.location))
        else:
            print("%s: %s / %s " % (j,varname,varvalue))


3716: Acacia binervata
7: Family / Fabaceae: Mimosoideae 
8: Group / D 
9: Life form / T 
10: Fireresponse / Sr 
11: Comments on regeneration / Resprouting form in northern tablelands (134) 
12: Resprout location / basal buds 
13: Seed storage / persistent soil 
14: Seed dispersal mechanism / a-ant / References!C56
18: Dormancy / hard seed coat 
19: Germination cue / heat / References!C94
22: Post-fire recruitment / prolific / References!C106
23: Establishment / I / References!A94
25: Flowering time / Aug-Nov 
26: Primary juvenile period / 5 / References!C36
27: Secondary juvenile period / References!C36 / References!C36
28: Seed set / 6 / References!C36
29: Seed-bank developed / 10 / References!C36


## Search for a species code
Here we try to locate the species code and then return the values for that row:

In [125]:
for cell in ws['A']:
    if(cell.value is not None): #We need to check that the cell is not empty.
        if 'Eryngium vesiculosum' in cell.value: #Check if the value of the cell contains the text 'Table'
            print('Found header with name: {} at row: {} and column: {}. In cell {}'.format(cell.value,cell.row,cell.column,cell))

Found header with name: Eryngium vesiculosum at row: 1123 and column: 1. In cell <Cell 'SpeciesData'.A1123>


In [126]:
# for j in 17
i=1123
spname=ws.cell(row=i, column=1).value
spcode=ws.cell(row=i, column=2).value
print("%s: %s" %(spcode,spname))
    
for j in range(3,50):
    varname=ws.cell(row=2, column=j).value
    varvalue=ws.cell(row=i, column=j).value
    varref=ws.cell(row=i, column=j).hyperlink
    if varvalue is not None:
        if varref is not None:
            print("%s: %s / %s / %s" % (j,varname,varvalue, varref.location))
        else:
            print("%s: %s / %s " % (j,varname,varvalue))


1117: Eryngium vesiculosum
7: Family / Apiaceae 
8: Group / D 
9: Life form / H 
10: Fireresponse / R 
25: Flowering time / Dec-Mar 
27: Secondary juvenile period / <1 / References!C80
31: Life span / short / References!C2
37: NC / - 
38: CC / - 
39: SC / - 
40: NT / 1 
41: CT / 1 
42: ST / 1 
43: NWS / 1 
44: CWS / - 
45: SWS / 1 
46: NWP / - 
47: SWP / - 
48: NFWP / - 
49: SFWP / - 


## Populate the 'blue table'
Select one column, translate the values to the accepted range and update entries into the table

In [25]:
from configparser import ConfigParser

filename = repodir / 'secrets' / 'database.ini'
section = 'aws-lght-sl'

# create a parser
parser = ConfigParser()
# read config file
parser.read(filename)

# get section, default to postgresql
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

In [26]:
import psycopg2
params = db

In [112]:
from psycopg2.extras import DictCursor
conn = psycopg2.connect(**params)

#ON CONFLICT ON CONSTRAINT traits_ref_fkey DO NOTHING 
qry = "SELECT \"currentScientificNameCode\",\"currentScientificName\" from species.caps"
cur = conn.cursor(cursor_factory=DictCursor)
cur.execute(qry)
valid = cur.fetchall()
cur.close()

if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.


In [115]:
sppnames=pd.DataFrame(valid)


In [116]:
sppnames

,0,1
0,1826,Lepidium oxytrichum
1,8683,Eucalyptus williamsiana
2,4251,Melaleuca glomerata
3,1094,Actinotus helianthi
4,1104,Apium prostratum
...,...,...
15727,15221,Pterostylis crebra
15728,15222,Trachymene sp. NSW Northern Tablelands
15729,15223,Pittosporum sp. Coffs Harbour
15730,15224,Adansonia gregorii


In [55]:
import pandas as pd
conn = psycopg2.connect(**params)
result_df = pd.read_sql_query(qry, conn)
if conn is not None:
    conn.close()
    print('Database connection closed.')

/Users/jferrer/miniconda3/envs/jptr/lib/python3.10/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Database connection closed.


In [56]:
result_df['speciesID']

0         2358
1         2359
2         2360
3         2361
4         2362
         ...  
15727    24289
15728    24291
15729    24292
15730    24293
15731    24294
Name: speciesID, Length: 15732, dtype: int64

In [58]:
2358 in result_df.speciesID.values

True

In [74]:
2358 in pd.DataFrame(valid)[0]

True

### Survival traits


In [144]:
ws.cell(row=2, column=67).value

'Additional fire response data'

In [150]:
ws = wb['SpeciesData']
va_groups = wb['VA Groups']
references = wb['References']

In [155]:
references.cell(row=1, column=20).value

'NFRR Reference details'

In [160]:
NFRR_refs=references[2]
NFRR_refs

(<Cell 'References'.A2>,
 <Cell 'References'.B2>,
 <Cell 'References'.C2>,
 <Cell 'References'.D2>,
 <Cell 'References'.E2>,
 <Cell 'References'.F2>,
 <Cell 'References'.G2>,
 <Cell 'References'.H2>,
 <Cell 'References'.I2>,
 <Cell 'References'.J2>,
 <Cell 'References'.K2>,
 <Cell 'References'.L2>,
 <Cell 'References'.M2>,
 <Cell 'References'.N2>,
 <Cell 'References'.O2>,
 <Cell 'References'.P2>,
 <Cell 'References'.Q2>,
 <Cell 'References'.R2>,
 <Cell 'References'.S2>,
 <Cell 'References'.T2>,
 <Cell 'References'.U2>,
 <Cell 'References'.V2>,
 <Cell 'References'.W2>,
 <Cell 'References'.X2>,
 <Cell 'References'.Y2>,
 <Cell 'References'.Z2>,
 <Cell 'References'.AA2>,
 <Cell 'References'.AB2>,
 <Cell 'References'.AC2>,
 <Cell 'References'.AD2>,
 <Cell 'References'.AE2>,
 <Cell 'References'.AF2>,
 <Cell 'References'.AG2>,
 <Cell 'References'.AH2>,
 <Cell 'References'.AI2>,
 <Cell 'References'.AJ2>,
 <Cell 'References'.AK2>,
 <Cell 'References'.AL2>,
 <Cell 'References'.AM2>,
 <Cell 'Refe

In [149]:
from psycopg2.extensions import AsIs

# for i in 3:3088
# for j in 10
row_count = ws.max_row
row_min=3
row_max=row_count
i=3
j=10
varname=ws.cell(row=2, column=j).value
print(varname)

switcher={
    "S": "none",
    "Sr": "few",
    "S/R": "half",
    "Rs": "most",
    "R": "all"
}
row_min=3
row_max=10

# connect to the PostgreSQL server
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**params)
cur = conn.cursor()


for i in range(row_min,row_max):
    record={"proximate_source":"NSWFFRD", "raw_value":[varname],"original_sources":list()}
    spname=ws.cell(row=i, column=1).value
    spcode=ws.cell(row=i, column=2).value
    varvalue=ws.cell(row=i, column=j).value
    
    NFRRdata=ws.cell(row=i, column=66).value
    
    AdditionalData=ws.cell(row=i, column=67).value
    varref=ws.cell(row=i, column=j).hyperlink
    if varvalue is not None:
        record["raw_value"].append(varvalue)
        transvalue=switcher.get(varvalue, "unknown")
        record["norm_value"]=transvalue
        if spcode is not None:
            record["species_code"]=spcode
        if spname is not None:
            record["species"]=spname
        if varref is not None:
            record["original_sources"].append(varref)
        if NFRRdata is not None:
            record["original_sources"].append('NFRR' % NFRRdata)
        if AdditionalData is not None:
            record["original_sources"].append(AdditionalData)
        if len(record["original_sources"])==0:
            record["original_sources"]=None

        print(record)
        
        insert_statement = 'insert into litrev.resprouting (%s) values %s ON CONFLICT DO NOTHING'
        #print(i)
        #print(cur.mogrify(insert_statement, (AsIs(','.join(record.keys())), tuple(record.values()))))
        cur.execute(insert_statement, (AsIs(','.join(record.keys())), tuple(record.values())))

conn.commit()
cur.close()
#print("total number of lines updated: %s" % totalupdates)

if conn is not None:
    conn.close()
    print('Database connection closed.')

 

Fireresponse
Connecting to the PostgreSQL database...
{'proximate_source': 'NSWFFRD', 'raw_value': ['Fireresponse', 'S'], 'original_sources': ['8CN'], 'norm_value': 'none', 'species_code': '7184', 'species': 'Abutilon otocarpum'}
{'proximate_source': 'NSWFFRD', 'raw_value': ['Fireresponse', 'S'], 'original_sources': ['8BD', 'II-95b'], 'norm_value': 'none', 'species_code': '3632', 'species': 'Abutilon oxycarpum'}
{'proximate_source': 'NSWFFRD', 'raw_value': ['Fireresponse', 'S'], 'original_sources': ['8C'], 'norm_value': 'none', 'species_code': '3698', 'species': 'Acacia acanthoclada'}
{'proximate_source': 'NSWFFRD', 'raw_value': ['Fireresponse', 'S'], 'original_sources': ['8W 8WA', 'II-38d'], 'norm_value': 'none', 'species_code': '3699', 'species': 'Acacia acinacea'}
{'proximate_source': 'NSWFFRD', 'raw_value': ['Fireresponse', 'S'], 'original_sources': ['IId-38d; III-67'], 'norm_value': 'none', 'species_code': '3700', 'species': 'Acacia aculeatissima'}
{'proximate_source': 'NSWFFRD', 

In [117]:


totalupdates=0
includedcodes=list()
excludedcodes=list()

#ON CONFLICT ON CONSTRAINT traits_ref_fkey DO NOTHING 
qrystr= "INSERT INTO litrev.resprouting(species,species_code,resprouting,proximate_source) values(%s,%s,%s) ON CONFLICT DO NOTHING"
for i in range(3,row_count):
    record={"proximate_source":"NSWFFRD", "raw value":(varname)}
    spname=ws.cell(row=i, column=1).value
    spcode=ws.cell(row=i, column=2).value
    varvalue=ws.cell(row=i, column=j).value
    varref=ws.cell(row=i, column=j).hyperlink
    if varvalue is not None:
        if spcode is not None:
            record["species_code"]=spcode
        
        if spname is not None:
            record["species"]=spname
        includedcodes.append(spcode)
            transvalue=switcher.get(varvalue, "unknown")
            cur.execute(qrystr, (spname,int(spcode),transvalue))
            updated_rows = cur.rowcount
            if updated_rows > 0:
                totalupdates=totalupdates+1



        

Fireresponse
Connecting to the PostgreSQL database...
total number of lines updated: 3049
Database connection closed.


In [118]:
print("%s excluded / %s included" % (len(set(excludedcodes)), len(set(includedcodes))))

2998 excluded / 0 included


In [77]:
int(spcode)

6354

In [119]:
j=12
varname=ws.cell(row=2, column=j).value
print(varname)


switcher={
    "epicormic": "epicormic",
    "stem buds": "epicormic",
    "apical": "apical",
    "lignotuber": "lignotuber", 
    "rootstock": "lignotuber", 
    "root stock": "lignotuber",
    "basal": "basal",
    "basal buds": "basal",
    "coppice": "basal",
    "tuber": "tuber",
    "taproot": "tuber",
    "tussock": "tussock",
    "rhizome": "short rhizome",
    "root sucker": "long rhizome or root sucker",
    "root suckers": "long rhizome or root sucker",
    "rootsucker": "long rhizome or root sucker",
    "root buds": "long rhizome or root sucker",
    "stolon": "stolon",
    "stolons": "stolon"
}
# connect to the PostgreSQL server
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**params)
cur = conn.cursor()

qrystr= "INSERT INTO litrev.survival_traits(species,species_code,regenerative_organ) values(%s,%s,%s) ON CONFLICT (species,species_code) DO UPDATE SET regenerative_organ=EXCLUDED.regenerative_organ"
updated_rows=0
for i in range(3,row_count):
    spname=ws.cell(row=i, column=1).value
    spcode=ws.cell(row=i, column=2).value
    varvalue=ws.cell(row=i, column=j).value
    varref=ws.cell(row=i, column=j).hyperlink
    if varvalue is not None:
        transvalue=switcher.get(varvalue, "other")
        if spcode.isnumeric():
            cur.execute(qrystr, (spname,int(spcode),transvalue))
            updated_rows = updated_rows + cur.rowcount

print("Total of %s rows updated" % (updated_rows))
conn.commit()
cur.close()

if conn is not None:
    conn.close()
    print('Database connection closed.')

        

Resprout location
Connecting to the PostgreSQL database...
Total of 1242 rows updated
Database connection closed.


In [120]:
j=31
varname=ws.cell(row=2, column=j).value
print(varname)

# connect to the PostgreSQL server
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**params)
cur = conn.cursor()

qrystr= "INSERT INTO litrev.survival_traits(species,species_code,standing_plant_longevity) values(%s,%s,%s) ON CONFLICT (species,species_code) DO UPDATE SET standing_plant_longevity=EXCLUDED.standing_plant_longevity"
updated_rows=0
includedcodes=list()
excludedcodes=list()
for i in range(3,row_count):
    spname=ws.cell(row=i, column=1).value
    spcode=ws.cell(row=i, column=2).value
    varvalue=ws.cell(row=i, column=j).value
    varref=ws.cell(row=i, column=j).hyperlink
    if varvalue is not None:
        if spcode.isnumeric():
            if (isinstance(varvalue, str) and varvalue.isnumeric()) or isinstance(varvalue,int):
                #print("%s : %s (numeric)" % (spcode,varvalue))
                cur.execute(qrystr, (spname,int(spcode),int(varvalue)))
                updated_rows = updated_rows + cur.rowcount
                includedcodes.append(spcode)
            else:
                transvalue=varvalue.split("-")
                if len(transvalue)>1 and transvalue[1].isnumeric:
                    print("%s : %s (orig. string %s)" % (spcode,transvalue[1],varvalue))
                    try:
                        cur.execute(qrystr, (spname,int(spcode),int(transvalue[1])))
                        updated_rows = updated_rows + cur.rowcount
                        includedcodes.append(spcode)
                    except:
                        print("This did not work > %s : %s (orig. string %s)" % (spcode,transvalue[1],varvalue))
                        excludedcodes.append(spcode)                
                else:
                    print("%s : %s (string)" % (spcode,varvalue))
                    excludedcodes.append(spcode)

print("Total of %s rows updated" % (updated_rows))
print("%s excluded / %s included" % (len(set(excludedcodes)), len(set(includedcodes))))
conn.commit()
cur.close()

if conn is not None:
    conn.close()
    print('Database connection closed.')


Life span
Connecting to the PostgreSQL database...
3632 : 10 (orig. string 4-10)
3710 : 30 (orig. string 20-30)
3717 : 100 (orig. string 50-100)
8601 : 40 (orig. string c. 20-40)
3747 : short? (string)
3762 : 30 (orig. string 15-30)
3768 : 50? (string)
3771 : 20 (orig. string 5-20)
3777 : 60 (orig. string 25-60)
3780 : 30 (orig. string c. 20-30)
3782 : long (string)
3790 : <10 (string)
3792 : <50? (string)
3797 : <15? (string)
9983 : 25 (orig. string c. 15-25)
3807 : >10 (string)
3814 : 15 (orig. string c.10-15)
10790 : 60 (orig. string 20-60)
10791 : 30 (orig. string 5-30)
3821 : c. 20 (string)
3822 : 4y (orig. string c.8; high attrition at 2-4y)
This did not work > 3822 : 4y (orig. string c.8; high attrition at 2-4y)
3823 : 20 (84) / short (1) (string)
3824 : 100 (M) (orig. string 50 (1) / 90 (CT) / 50-100 (M))
This did not work > 3824 : 100 (M) (orig. string 50 (1) / 90 (CT) / 50-100 (M))
3832 : 25 (orig. string 15-25)
3834 : 25 (1) / c. 30 (101) (orig. string 5-25 (1) / c. 30 (101)

### Other traits

In [39]:
j=13
varname=ws.cell(row=2, column=j).value
print(varname)


switcher={
    "canopy":"canopy",
    "canopy - transient":"canopy",
    "persistent":"soil-persistent",
    "peristent":"soil-persistent",
    "persistent soil":"soil-persistent", 
    "a-persistent soil":"soil-persistent", 
    "soil":"soil-persistent",
    "transient":"transient", 
    "none":"transient",
    "shed at maturity":"transient", 
    "viviparous":"transient",
    "canopy / released at maturity":"transient",
    "canopy / regularly without fire":"transient",
    "not canopy":"non-canopy",
}

# connect to the PostgreSQL server
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**params)
qrystr= "INSERT INTO litrev.traits(species,species_code,seedbank_type) values(%s,%s,%s) ON CONFLICT (species,species_code) DO UPDATE SET seedbank_type=EXCLUDED.seedbank_type"
updated_rows = 0

for i in range(1300,2800):
    spname=ws.cell(row=i, column=1).value
    spcode=ws.cell(row=i, column=2).value
    varvalue=ws.cell(row=i, column=j).value
    varref=ws.cell(row=i, column=j).hyperlink
    if varvalue is not None:
        transvalue=switcher.get(varvalue, "other")
        #if varref is not None:
        #    print("%s: %s / (%s) %s / %s" % (spcode,spname,varvalue,transvalue, varref.location))
        #else:
        #    print("%s: %s / (%s) %s " % (spcode,spname,varvalue,transvalue))
        if spcode.isnumeric():
            cur = conn.cursor()
            cur.execute(qrystr, (spname,int(spcode),transvalue))
            updated_rows = updated_rows + cur.rowcount
            conn.commit()
            cur.close()
print("Total of %s rows updated" % (updated_rows))


if conn is not None:
    conn.close()
    print('Database connection closed.')
            

Seed storage
Connecting to the PostgreSQL database...
Total of 2660 rows updated
Database connection closed.


In [42]:
j=22
varname=ws.cell(row=2, column=j).value
print(varname)

switcher={
    "prolific":"abundant",
    "abundant":"abundant",
    "common":"abundant",
    "high":"abundant",
    "substantial":"abundant",
    "vigourous":"abundant",
    "1-4 seedlings/m2":"abundant",
    "no":"absent",
    "none observed":"absent"
}

# connect to the PostgreSQL server
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**params)
qrystr= "INSERT INTO litrev.traits(species,species_code,postfire_seedling_recruitment) values(%s,%s,%s) ON CONFLICT (species,species_code) DO UPDATE SET postfire_seedling_recruitment=EXCLUDED.postfire_seedling_recruitment"
cur = conn.cursor()
updated_rows = 0
for i in range(3,3000):
    spname=ws.cell(row=i, column=1).value
    spcode=ws.cell(row=i, column=2).value
    varvalue=ws.cell(row=i, column=j).value
    varref=ws.cell(row=i, column=j).hyperlink
    if varvalue is not None:
        transvalue=switcher.get(varvalue, "present")
        #if varref is not None:
        #    print("%s: %s / (%s) %s / %s" % (spcode,spname,varvalue,transvalue, varref.location))
        #else:
        #    print("%s: %s / (%s) %s " % (spcode,spname,varvalue,transvalue))
        if spcode.isnumeric():
            cur.execute(qrystr, (spname,int(spcode),transvalue))
            updated_rows = updated_rows + cur.rowcount
            
conn.commit()
cur.close()            
print("Total of %s rows updated" % (updated_rows))

if conn is not None:
    conn.close()
    print('Database connection closed.')
        

Post-fire recruitment
Connecting to the PostgreSQL database...
Total of 590 rows updated
Database connection closed.
